# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***@rodrigo32798866***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [3]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @rodrigo32798866

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [4]:
#Produto escolhido:
produto = 'trident'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [5]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [6]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [18]:
%matplotlib inline
from mpmath import mpf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
plt.style.use('ggplot')


#dados_Treinamento = pd.read_excel('trident.xlsx', "Treinamento", sep=',')
dados_Teste = pd.read_excel('trident.xlsx', "Teste", sep=',')

In [37]:
#pegando os dados do treinamento
dados_Treinamento = pd.read_excel('trident.xlsx', sheetname="Treinamento")
dados_Treinamento = dados_Treinamento.loc[:,["Treinamento", "sentimento"]]


In [38]:
#Probabilidades Treinamento relevante
# valuecounts_Treinamento.loc["neutro"]

In [39]:
#probabilidade Treinamento irrelevante
# valuecounts_Treinamento.loc["irrelevante"]

In [40]:
#juntando tudo em um tweet gigante
soma_Treinamento = np.sum(dados_Treinamento.Treinamento + " ")
soma_Treinamentoirrelevante = np.sum(dados_Treinamento[dados_Treinamento.sentimento == "irrelevante"].Treinamento + " ")
soma_Treinamentoneutro = np.sum(dados_Treinamento[dados_Treinamento.sentimento == "neutro"].Treinamento + " ")

In [41]:
#Tirar toda os caracteres necessarios
import string

excluir= set(string.punctuation)
def excluirpontos(soma):
    soma = soma.split()
    lista_palavrastweets = []
    for r in soma:
        for i in r:
            if i in excluir:
                r=r.replace(i, '')
        lista_palavrastweets.append(r)
    return lista_palavrastweets

treinamentototal = excluirpontos(soma_Treinamento)
treinamentoir = excluirpontos(soma_Treinamentoirrelevante)
treinamentone = excluirpontos(soma_Treinamentoneutro)

In [48]:
treinamentototal= pd.Series(treinamentototal)
valuecounts_treinamentototal= treinamentototal.value_counts()

treinamentoir= pd.Series(treinamentoir)
valuecounts_treinamentoir= treinamentoir.value_counts()

treinamentone= pd.Series(treinamentone)
valuecounts_treinamentone= treinamentone.value_counts()

In [60]:
prob_palavras_ir= []
prob_palavra= (valuecounts_treinamentoir + 1)/(len(valuecounts_treinamentototal) + len(valuecounts_treinamentoir))
prob_palavras_ir.append(prob_palavra)
print(prob_palavras_ir)

[trident               0.071541
rt                    0.034193
um                    0.033666
de                    0.033140
e                     0.029984
                      0.026302
que                   0.025776
na                    0.024724
eu                    0.022094
o                     0.019463
a                     0.015781
não                   0.014729
festa                 0.014203
é                     0.014203
com                   0.013151
tem                   0.013151
no                    0.010521
pra                   0.010521
mavi                  0.010521
braga18mavi           0.010521
parte                 0.009995
meu                   0.008943
os                    0.008417
mas                   0.007891
por                   0.007365
da                    0.007365
me                    0.006839
dos                   0.006839
uma                   0.006839
do                    0.006839
                        ...   
compramos             0.001052
mal    

In [61]:
prob_palavras_ne= []
prob_palavra2= (valuecounts_treinamentone + 1)/(len(valuecounts_treinamentototal) + len(valuecounts_treinamentone))
prob_palavras_ne.append(prob_palavra2)
print(prob_palavras_ne)

[trident               0.097262
rt                    0.056196
um                    0.044669
é                     0.033862
tem                   0.033862
que                   0.033141
de                    0.030259
não                   0.029539
no                    0.026657
abrindo               0.024496
amigos                0.024496
meio                  0.024496
dos                   0.024496
igual                 0.023775
ciúmes                0.023055
fala                  0.023055
mas                   0.023055
vc                    0.020893
eu                    0.018732
hmdaoras              0.018732
httpstcoq76wbpx3u1    0.018732
e                     0.010807
se                    0.010086
zuadavida             0.010086
por                   0.010086
canela                0.009366
quero                 0.009366
quando                0.009366
pegar                 0.008646
olhando               0.007925
                        ...   
ganhei                0.001441
tocops 

In [62]:
#pegando os dados do teste
dados_Teste = pd.read_excel('trident.xlsx', sheetname="Teste")
dados_Teste = dados_Teste.loc[:,["Teste", "sentimento"]]

In [63]:
lista_teste = dados_Teste.Teste
#tirando a pontuacao
import string
excluir= set(string.punctuation)

def excluirpontos(soma):
    lista_tweets_em_palavras=[]
    for s in range(len(soma)):
        frase = soma[s].split()
        lista_palavras=[]
        
        for r in frase:
            for l in r:
                if l in excluir:
                    r=r.replace(l,'')
            
            lista_palavras.append(r)
        lista_tweets_em_palavras.append(lista_palavras)
        
    return lista_tweets_em_palavras

lista_pal_tweets= excluirpontos(lista_teste)

In [ ]:
def prob():
    lista_probabilidades=[]
    for 

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [17]:
! explorer .


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
